# TfidfVectorizer Explanation
Convert a collection of raw documents to a matrix of TF-IDF features

TF-IDF where TF means term frequency, and IDF means Inverse Document frequency.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = ['Hello Kushal Bhavsar here, I love machine learning','Welcome to the Machine learning hub' ]

In [2]:
vect = TfidfVectorizer()

In [3]:
vect.fit(text)

TfidfVectorizer()

In [4]:
## TF will count the frequency of word in each document. and IDF 
print(vect.idf_)

[1.40546511 1.40546511 1.40546511 1.40546511 1.40546511 1.
 1.40546511 1.         1.40546511 1.40546511 1.40546511]


In [5]:
print(vect.vocabulary_)

{'hello': 1, 'kushal': 4, 'bhavsar': 0, 'here': 2, 'love': 6, 'machine': 7, 'learning': 5, 'welcome': 10, 'to': 9, 'the': 8, 'hub': 3}


### A words which is present in all the data, it will have low IDF value. With this unique words will be highlighted using the Max IDF values.

In [6]:
example = text[0]
example

'Hello Kushal Bhavsar here, I love machine learning'

In [7]:
example = vect.transform([example])
print(example.toarray())

[[0.4078241  0.4078241  0.4078241  0.         0.4078241  0.29017021
  0.4078241  0.29017021 0.         0.         0.        ]]


### Here, 0 is present in the which indexed word, which is not available in given sentence.

## PassiveAggressiveClassifier

### Passive: if correct classification, keep the model; Aggressive: if incorrect classification, update to adjust to this misclassified example.

Passive-Aggressive algorithms are generally used for large-scale learning. It is one of the few ‘online-learning algorithms‘. In online machine learning algorithms, the input data comes in sequential order and the machine learning model is updated step-by-step, as opposed to batch learning, where the entire training dataset is used at once. This is very useful in situations where there is a huge amount of data and it is computationally infeasible to train the entire dataset because of the sheer size of the data. We can simply say that an online-learning algorithm will get a training example, update the classifier, and then throw away the example.

## Let's start the work

In [8]:
import pandas as pd

In [9]:
dataframe = pd.read_csv('news.csv')
dataframe.head()

,text,label
0,Bitcoin steadies near lows hit after fall of S...,REAL
1,Crypto bank Silvergate announces liquidation a...,REAL
2,US stocks rise as investors assess softer empl...,REAL
3,Silvergate cierra: adiós al prestamista que se...,REAL
4,Nostr: Eine Alternative zu Twitter – und Mastodon,REAL


In [10]:
x = dataframe['text']
y = dataframe['label']

In [11]:
x

0       Bitcoin steadies near lows hit after fall of S...
1       Crypto bank Silvergate announces liquidation a...
2       US stocks rise as investors assess softer empl...
3       Silvergate cierra: adiós al prestamista que se...
4       Nostr: Eine Alternative zu Twitter – und Mastodon
                              ...                        
6480    The State Department told the Republican Natio...
6481    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6482     Anti-Trump Protesters Are Tools of the Oligar...
6483    ADDIS ABABA, Ethiopia —President Obama convene...
6484    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6485, dtype: object

In [12]:
y

0       REAL
1       REAL
2       REAL
3       REAL
4       REAL
        ... 
6480    REAL
6481    FAKE
6482    FAKE
6483    REAL
6484    REAL
Name: label, Length: 6485, dtype: object

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

5528    FAKE
3460    REAL
5406    FAKE
5239    FAKE
6193    FAKE
        ... 
4931    REAL
3264    REAL
1653    REAL
2607    FAKE
2732    FAKE
Name: label, Length: 5188, dtype: object

In [15]:
y_train

5528    FAKE
3460    REAL
5406    FAKE
5239    FAKE
6193    FAKE
        ... 
4931    REAL
3264    REAL
1653    REAL
2607    FAKE
2732    FAKE
Name: label, Length: 5188, dtype: object

In [16]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

* max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
* max_df = 25 means "ignore terms that appear in more than 25 documents".

In [17]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [18]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.91%


In [19]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[559  39]
 [ 40 659]]


In [20]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [21]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['REAL']


In [22]:
fake_news_det("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [23]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [24]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [25]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [26]:
fake_news_det1("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [27]:
fake_news_det1("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.""")

['REAL']


In [28]:
fake_news_det('''U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.''')

['REAL']
